In [1]:
import json
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

In [2]:
#Tesla - Annual Revenue
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/income-statement?freq=A"
driver = webdriver.Chrome(service=Service('./chromedriver.exe'))
driver.set_window_size(1824,1026)
driver.get(url)

table = driver.find_elements(By.ID,"jqxgrid")[0]
table.location_once_scrolled_into_view

bar = driver.find_elements(By.ID,"jqxScrollThumbhorizontalScrollBarjqxgrid")[0]
action = ActionChains(driver)
action.click_and_hold(bar).move_by_offset(380,0).release().perform()

soup = BeautifulSoup(driver.page_source, "html.parser")
yearly = soup.find_all(style="text-overflow: ellipsis; cursor: default;")[2:]
revenue = soup.find_all(style="font-weight:bold; text-align:right; margin-top:5px; margin-right:4px;")[:len(yearly)]
driver.close()

d_yearly = {"Year": (int(yearly[i].text[:4]) for i in range(len(yearly))), "Annual Revenue (Millions of US $)": (float(revenue[i].text.replace("$","").replace(",","")) for i in range(len(revenue)))}
df_yearly = pd.DataFrame(d_yearly)
df_Tesla_yearly = df_yearly[::-1]

In [3]:
#Tesla - Quarterly Revenue
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/income-statement?freq=Q"
driver = webdriver.Chrome(service=Service('./chromedriver.exe'))
driver.set_window_size(1824,1026)
driver.get(url)

table = driver.find_elements(By.ID,"jqxgrid")[0]
table.location_once_scrolled_into_view

soup = BeautifulSoup(driver.page_source, "html.parser")
temp_quarterly = soup.find_all(style="text-overflow: ellipsis; cursor: default;")[2:]
quarterly = [i.text for i in temp_quarterly]
temp_revenue = soup.find_all(style="font-weight:bold; text-align:right; margin-top:5px; margin-right:4px;")[:len(quarterly)]
revenue = [i.text for i in temp_revenue]

bar = driver.find_elements(By.ID,"jqxScrollThumbhorizontalScrollBarjqxgrid")[0]
action = ActionChains(driver)
for i in range(int((int(quarterly[0][:4])-2008)*4/15)) :
    action.click_and_hold(bar).move_by_offset(380,0).release().perform()
    soup = BeautifulSoup(driver.page_source, "html.parser")
    temp_quarterly = soup.find_all(style="text-overflow: ellipsis; cursor: default;")[2:]
    temp_revenue = soup.find_all(style="font-weight:bold; text-align:right; margin-top:5px; margin-right:4px;")[:len(temp_quarterly)]
    for j in temp_quarterly :
        if j.text in quarterly :
            continue
        else :
            quarterly.append(j.text)
            revenue.append(temp_revenue[temp_quarterly.index(j)].text)
revenue.pop(-1)
revenue.insert(-2,str(round(df_Tesla_yearly["Annual Revenue (Millions of US $)"].iloc[0]-float(revenue[-1].replace("$","").replace(",",""))-float(revenue[-2].replace("$","").replace(",","")),2)))
driver.close()

d_quarterly = {"Quarter": (date(int(quarterly[i][:4]),int(quarterly[i][5:7]),int(quarterly[i][8:])) for i in range(len(quarterly))), "Quarterly Revenue (Millions of US $)": (float(revenue[i].replace("$","").replace(",","")) for i in range(len(revenue)))}
df_quarterly = pd.DataFrame(d_quarterly)
df_Tesla_quarterly = df_quarterly[::-1]

In [4]:
#Tesla - Monthly Revenue
d = date(2009,5,1)
ml = []
for i in range(int(len(quarterly)*3)) :
    ml.append(date(int(d.strftime("%Y")),int(d.strftime("%m")),1) - timedelta(days=1))
    if int(ml[-1].strftime("%m"))+2 <=12 :
        d = date(int(ml[-1].strftime("%Y")),int(ml[-1].strftime("%m"))+2,1)
    elif int(ml[-1].strftime("%m"))+2 == 13 :
        d = date(int(ml[-1].strftime("%Y"))+1,1,1)
    else :
        d = date(int(ml[-1].strftime("%Y"))+1,2,1)

mrl = [i/3 for i in df_Tesla_quarterly["Quarterly Revenue (Millions of US $)"] for j in range(3)]

d_monthly = {"Month": ml, "Monthly Revenue (Millions of US $)": mrl}
df_Tesla_monthly = pd.DataFrame(d_monthly)

In [5]:
#IXIC - Daily
today = int(datetime(int(date.today().strftime("%Y")),int(date.today().strftime("%m")),int(date.today().strftime(f"%d"))+1).timestamp())

url = (f"https://query1.finance.yahoo.com/v8/finance/chart/%5EIXIC?symbol=%5EIXIC&period1=0&period2={today}&useYfid=true&interval=1d&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=c0iNuYsQEnn&corsDomain=finance.yahoo.com")
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"}
res = requests.get(url,headers=header)
content = json.loads(res.text)

date1 = content['chart']['result'][0]['timestamp']
date2 = [date.fromtimestamp(date1[j]).strftime(f"%Y-%m-%d") for j in range(len(date1))]
close = content['chart']['result'][0]['indicators']['quote'][0]['close']

d1 = {"date": date2, "close": close}
df_IXIC_daily = pd.DataFrame(d1)

In [6]:
#IXIC - Monthly
d = date(int(df_IXIC_daily["date"].iloc[0][:4]),int(df_IXIC_daily["date"].iloc[0][5:7])+1,1)
ml = []

if int(df_IXIC_daily["date"].iloc[-1][5:7]) > 2 :
    ml_len = (int(df_IXIC_daily["date"].iloc[-1][:4])-int(df_IXIC_daily["date"].iloc[0][:4]))*12+(int(df_IXIC_daily["date"].iloc[-1][5:7])-int(df_IXIC_daily["date"].iloc[0][5:7]))
elif int(df_IXIC_daily["date"].iloc[-1][5:7]) == 1 :
    ml_len = (int(df_IXIC_daily["date"].iloc[-1][:4])-int(df_IXIC_daily["date"].iloc[0][:4]))*12+11
else :
    ml_len = (int(df_IXIC_daily["date"].iloc[-1][:4])-int(df_IXIC_daily["date"].iloc[0][:4])+1)*12

for i in range(ml_len) :
    ml.append(date(int(d.strftime("%Y")),int(d.strftime("%m")),1) - timedelta(days=1))
    if int(ml[-1].strftime("%m"))+2 <=12 :
        d = date(int(ml[-1].strftime("%Y")),int(ml[-1].strftime("%m"))+2,1)
    elif int(ml[-1].strftime("%m"))+2 == 13 :
        d = date(int(ml[-1].strftime("%Y"))+1,1,1)
    else :
        d = date(int(ml[-1].strftime("%Y"))+1,2,1)

cml = []
for i in range(len(ml)):
    cmm = df_IXIC_daily[(df_IXIC_daily["date"]>=str(date(int(ml[i].strftime("%Y")),int(ml[i].strftime("%m")),1)))&(df_IXIC_daily["date"]<=str(ml[i]))]
    cml.append(sum(cmm["close"])/len(cmm))

d2 = {"month": ml, "close means": cml}
df_IXIC_monthly = pd.DataFrame(d2)

In [7]:
#IXIC - Quarterly
if len(df_IXIC_monthly)%3 == 2 :
    ql = df_IXIC_monthly["month"][1::3]
else :
    ql = np.append(np.array(df_IXIC_monthly["month"][1::3]),np.array(df_IXIC_monthly["month"].iloc[-1]))


cml_fill_1 = np.append(np.array([0]), np.array(df_IXIC_monthly["close means"]))

if len(cml_fill_1)%3 == 2 :
    cml_fill_2 = np.append(cml_fill_1, np.array([0]))
elif len(cml_fill_1)%3 == 1 :
    cml_fill_2 = np.append(cml_fill_1, np.array([0,0]))
else :
    pass

cml_fill_3 = np.split(cml_fill_2, len(cml_fill_2)/3)
cml = [(cml_fill_3[i].sum()/np.count_nonzero(cml_fill_3[i])) for i in range(len(cml_fill_3))]

d3 = {"quarter": ql, "close means": cml}
df_IXIC_quarterly = pd.DataFrame(d3)

In [8]:
#IXIC - Yearly
yl = [i for i in range(int(df_IXIC_daily["date"].iloc[0][:4]), int(df_IXIC_daily["date"].iloc[-1][:4]))]

cml = []
for i in yl :
    cmr = df_IXIC_daily[(df_IXIC_daily["date"]>=f"{i}-01-01")&(df_IXIC_daily["date"]<=f"{i}-12-31")]["close"]
    cml.append(cmr.sum()/len(cmr))

d4 = {"year": yl, "close means": cml}
df_IXIC_yearly = pd.DataFrame(d4)

In [9]:
#CL - Daily
today = int(datetime(int(date.today().strftime("%Y")),int(date.today().strftime("%m")),int(date.today().strftime(f"%d"))+1).timestamp())

url = (f"https://query1.finance.yahoo.com/v8/finance/chart/CL=F?symbol=CL%3DF&period1=0&period2={today}&useYfid=true&interval=1d&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=c0iNuYsQEnn&corsDomain=finance.yahoo.com")
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"}
res = requests.get(url,headers=header)
content = json.loads(res.text)

date1 = content['chart']['result'][0]['timestamp']
date2 = [date.fromtimestamp(date1[j]).strftime(f"%Y-%m-%d") for j in range(len(date1))]
close = content['chart']['result'][0]['indicators']['quote'][0]['close']

d1 = {"date": date2, "close": close}
df_CL_daily = pd.DataFrame(d1).dropna()

In [10]:
#CL - Monthly
d = date(int(df_CL_daily["date"].iloc[0][:4]),int(df_CL_daily["date"].iloc[0][5:7])+1,1)
ml = []

if int(df_CL_daily["date"].iloc[-1][5:7]) > 2 :
    ml_len = (int(df_CL_daily["date"].iloc[-1][:4])-int(df_CL_daily["date"].iloc[0][:4]))*12+(int(df_CL_daily["date"].iloc[-1][5:7])-int(df_CL_daily["date"].iloc[0][5:7]))
elif int(df_CL_daily["date"].iloc[-1][5:7]) == 1 :
    ml_len = (int(df_CL_daily["date"].iloc[-1][:4])-int(df_CL_daily["date"].iloc[0][:4]))*12+11
else :
    ml_len = (int(df_CL_daily["date"].iloc[-1][:4])-int(df_CL_daily["date"].iloc[0][:4])+1)*12

for i in range(ml_len) :
    ml.append(date(int(d.strftime("%Y")),int(d.strftime("%m")),1) - timedelta(days=1))
    if int(ml[-1].strftime("%m"))+2 <=12 :
        d = date(int(ml[-1].strftime("%Y")),int(ml[-1].strftime("%m"))+2,1)
    elif int(ml[-1].strftime("%m"))+2 == 13 :
        d = date(int(ml[-1].strftime("%Y"))+1,1,1)
    else :
        d = date(int(ml[-1].strftime("%Y"))+1,2,1)

cml = []
for i in range(len(ml)):
    cmm = df_CL_daily[(df_CL_daily["date"]>=str(date(int(ml[i].strftime("%Y")),int(ml[i].strftime("%m")),1)))&(df_CL_daily["date"]<=str(ml[i]))]
    cml.append(sum(cmm["close"])/len(cmm))

d2 = {"month": ml, "close means": cml}
df_CL_monthly = pd.DataFrame(d2)

In [11]:
#CL - Quarterly
if len(df_CL_monthly)%3 == 2 :
    ql = df_CL_monthly["month"][1::3]
else :
    ql = np.append(np.array(df_CL_monthly["month"][1::3]),np.array(df_CL_monthly["month"].iloc[-1]))


cml_fill_1 = np.append(np.array([0]), np.array(df_CL_monthly["close means"]))

if len(cml_fill_1)%3 == 2 :
    cml_fill_2 = np.append(cml_fill_1, np.array([0]))
elif len(cml_fill_1)%3 == 1 :
    cml_fill_2 = np.append(cml_fill_1, np.array([0,0]))
else :
    pass

cml_fill_3 = np.split(cml_fill_2, len(cml_fill_2)/3)
cml = [(cml_fill_3[i].sum()/np.count_nonzero(cml_fill_3[i])) for i in range(len(cml_fill_3))]

d3 = {"quarter": ql, "close means": cml}
df_CL_quarterly = pd.DataFrame(d3)

In [12]:
#CL - Yearly
yl = [i for i in range(int(df_CL_daily["date"].iloc[0][:4]), int(df_CL_daily["date"].iloc[-1][:4]))]

cml = []
for i in yl :
    cmr = df_CL_daily[(df_CL_daily["date"]>=f"{i}-01-01")&(df_CL_daily["date"]<=f"{i}-12-31")]["close"]
    cml.append(cmr.sum()/len(cmr))

d4 = {"year": yl, "close means": cml}
df_CL_yearly = pd.DataFrame(d4)

In [13]:
#Household Income - Yearly
url = "https://www.deptofnumbers.com/income/us/"
soup = BeautifulSoup(requests.get(url).text, "html.parser")
yl = [int(i.text) for i in soup.find(id="table_588047").find_all("td", class_="col1")]
il = [int(i.text.replace("$","").replace(",","")) for i in soup.find(id="table_588047").find_all("td", class_="rstat")]
d = {"year": yl, "US$": il}
df_HI = pd.DataFrame(d)[::-1]

In [14]:
#Family Income - Yearly
url = "https://www.deptofnumbers.com/income/us/"
soup = BeautifulSoup(requests.get(url).text, "html.parser")
yl = [int(i.text) for i in soup.find(id="table_839916").find_all("td", class_="col1")]
il = [int(i.text.replace("$","").replace(",","")) for i in soup.find(id="table_839916").find_all("td", class_="rstat")]
d = {"year": yl, "US$": il}
df_FI = pd.DataFrame(d)[::-1]

In [15]:
#Capita Income - Yearly
url = "https://www.deptofnumbers.com/income/us/"
soup = BeautifulSoup(requests.get(url).text, "html.parser")
yl = [int(i.text) for i in soup.find(id="table_49011").find_all("td", class_="col1")]
il = [int(i.text.replace("$","").replace(",","")) for i in soup.find(id="table_49011").find_all("td", class_="rstat")]
d = {"year": yl, "US$": il}
df_CI = pd.DataFrame(d)[::-1]

In [16]:
#Vehicle Sales - Yearly, Quarterly, Monthly
url = "https://www.goodcarbadcar.net/usa-auto-industry-total-sales-figures/"
soup = BeautifulSoup(requests.get(url).text, "html.parser")

l = soup.find("table", id="table_3").find_all("td", style="")

yl = [int(i.text) for i in l[0::13]]
msl = [int(l[i].text.replace(",","")) for i in range(len(l)) if i%13 != 0]

ml = []
for i in yl :
    for j in range(2,13) :
        ml.append(date(i,j,1) - timedelta(days=1))
    ml.append(date(i+1,1,1) - timedelta(days=1))
ql = [ml[2::3][i] for i in range(int(np.count_nonzero(msl)/3))]


qsl = [np.array_split(msl,len(msl)/3)[i].sum() for i in range(int(np.count_nonzero(msl)/3))]
ysl = [np.array_split(msl,len(msl)/12)[i].sum() for i in range(int(len(msl)/12))]

dy = {"year": yl, "Sales": ysl}
df_VS_yearly = pd.DataFrame(dy)
dq = {"quarter": ql, "Sales": qsl}
df_VS_quarterly = pd.DataFrame(dq)
dm = {"month": ml, "Sales": msl}
df_VS_monthly = pd.DataFrame(dm)

In [17]:
#Final yearly data frame

kl = ["Year", "Tesla Annual Revenue (Millions)", "IXIC", "CL", "Household Income", "Family Income", "Capita Income", "Vehicle Sales"]

d1 = df_Tesla_yearly["Year"].reset_index(drop=True)
d2 = df_Tesla_yearly["Annual Revenue (Millions of US $)"].reset_index(drop=True)
d3 = df_IXIC_yearly[df_IXIC_yearly["year"]>=2009]["close means"].reset_index(drop=True)
d4 = df_CL_yearly[df_CL_yearly["year"]>=2009]["close means"].reset_index(drop=True)
d5 = df_HI[df_HI["year"]>=2009]["US$"].reset_index(drop=True)
d6 = df_FI[df_FI["year"]>=2009]["US$"].reset_index(drop=True)
d7 = df_CI[df_CI["year"]>=2009]["US$"].reset_index(drop=True)
d8 = df_VS_yearly[(df_VS_yearly["year"]>=2009)&(df_VS_yearly["year"]<=2021)]["Sales"].reset_index(drop=True)
vl = [d1, d2, d3, d4, d5, d6, d7, d8]

d_yearly = {kl[i]:vl[i] for i in range(len(kl))}
df_yearly = pd.DataFrame(d_yearly)

In [18]:
#Final quarterly data frame

kl = ["Quarter", "Tesla Quarterly Revenue (Millions)", "IXIC", "CL", "Vehicle Sales"]

d1 = df_Tesla_quarterly["Quarter"].reset_index(drop=True)
d2 = df_Tesla_quarterly["Quarterly Revenue (Millions of US $)"].reset_index(drop=True)
d3 = df_IXIC_quarterly[(df_IXIC_quarterly["quarter"]>=date(2009,6,30))&(df_IXIC_quarterly["quarter"]<=date(2022,9,30))]["close means"].reset_index(drop=True)
d4 = df_CL_quarterly[(df_CL_quarterly["quarter"]>=date(2009,6,30))&(df_CL_quarterly["quarter"]<=date(2022,9,30))]["close means"].reset_index(drop=True)
d5 = df_VS_quarterly[(df_VS_quarterly["quarter"]>=date(2009,6,30))&(df_VS_quarterly["quarter"]<=date(2022,9,30))]["Sales"].reset_index(drop=True)
vl = [d1, d2, d3, d4, d5]

d_quarterly = {kl[i]:vl[i] for i in range(len(kl))}
df_quarterly = pd.DataFrame(d_quarterly)

In [19]:
#Final monthly data frame

kl = ["Month", "Tesla Monthly Revenue (Millions)", "IXIC", "CL", "Vehicle Sales"]

d1 = df_Tesla_monthly["Month"].reset_index(drop=True)
d2 = df_Tesla_monthly["Monthly Revenue (Millions of US $)"].reset_index(drop=True)
d3 = df_IXIC_monthly[(df_IXIC_monthly["month"]>=date(2009,4,30))&(df_IXIC_monthly["month"]<=date(2022,9,30))]["close means"].reset_index(drop=True)
d4 = df_CL_monthly[(df_CL_monthly["month"]>=date(2009,4,30))&(df_CL_monthly["month"]<=date(2022,9,30))]["close means"].reset_index(drop=True)
d5 = df_VS_monthly[(df_VS_monthly["month"]>=date(2009,4,30))&(df_VS_monthly["month"]<=date(2022,9,30))]["Sales"].reset_index(drop=True)
vl = [d1, d2, d3, d4, d5]

d_monthly = {kl[i]:vl[i] for i in range(len(kl))}
df_monthly = pd.DataFrame(d_monthly)

In [20]:
yearly_list = [df_Tesla_yearly, df_IXIC_yearly, df_CL_yearly, df_HI, df_FI, df_CI, df_VS_yearly]
quarterly_list = [df_Tesla_quarterly, df_IXIC_quarterly, df_CL_quarterly, df_VS_quarterly]
monthly_list = [df_Tesla_monthly, df_IXIC_monthly, df_CL_monthly, df_VS_monthly]
daily_list = [df_IXIC_daily, df_CL_daily]
df_list = [df_yearly, df_quarterly, df_monthly]